In [3]:
from music21 import converter, instrument, note, chord
import glob
import pickle
import pandas as pd
import tqdm
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

notes = []
log = []

Using TensorFlow backend.
/Users/jean-baptistegourlet/opt/anaconda3/envs/ml37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jean-baptistegourlet/opt/anaconda3/envs/ml37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jean-baptistegourlet/opt/anaconda3/envs/ml37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dt

In [4]:
annees=[2002, 2004, 2006, 2008, 2009, 2011, 2013, 2015, 2017, 2018]
notes=[]
for x in annees:
    notesAnnees=pd.read_pickle(r'../data/results/notes_'+str(x))
    notes+=notesAnnees

print(len(notes))

8892971


In [14]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number)
                       for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(
        network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def create_network_allege(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        10,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(10, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(10))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model


def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=2,
              batch_size=1, callbacks=callbacks_list)

testing on sample of 1000 notes

In [15]:
notes = notes[0:1000]
# get amount of pitch names
n_vocab = len(set(notes))

In [16]:
#nombre de notes différentes à dispo
n_vocab

428

In [17]:
#exemples des 10 premieres notes
notes[0:10]

['B-3.F3.D3',
 'B-2.B-1',
 'C#3',
 'C#2',
 'B-3.F3.D3',
 'B-2.B-1',
 'D3.F3.B-2',
 'D3.B-2.F3',
 'F3.D3',
 'B-2']

In [18]:
network_input, network_output = prepare_sequences(notes, n_vocab)



In [19]:
#l'output a pour dimension notes_passées*nb de séquences
network_input.shape

(900, 100, 1)

In [ ]:
#categorique
network_output.shape

In [ ]:
#model = create_network(network_input, n_vocab)
model = create_network_allege(network_input, n_vocab)



In [ ]:
model.fit(network_input, network_output, epochs=2,
              batch_size=1)

In [ ]:
model.fit(network_input, network_output, epochs = 2)


In [ ]:
y_predicted = model.predict(network_input)


In [ ]:
y_predicted

In [ ]:
train(model, network_input, network_output)



In [20]:
train(model, network_input, network_output)



(900, 428)

In [21]:
#model = create_network(network_input, n_vocab)
model = create_network_allege(network_input, n_vocab)



In [22]:
model.fit(network_input, network_output, epochs=2,
              batch_size=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/2
900/900 [==============================] - 128s 143ms/step - loss: 9.7375
Epoch 2/2
900/900 [==============================] - 125s 139ms/step - loss: 9.8060


In [16]:
model.fit(network_input, network_output, epochs = 2)


ValueError: Error when checking target: expected dense_1 to have shape (1,) but got array with shape (428,)

In [ ]:
y_predicted = model.predict(network_input)


In [1]:
y_predicted

NameError: name 'y_predicted' is not defined

In [29]:
train(model, network_input, network_output)



NameError: name 'filepath' is not defined